In [ ]:
import os 
del os.environ['GDAL_DATA']

In [ ]:
import ee
import geemap
import numpy as np
import rasterio
from rasterio.mask import mask
from shapely.geometry import shape, box
import geopandas as gpd
from scripts import parameters as pm
from scripts.utils import update_progress

In [ ]:
ee.Initialize()

In [ ]:
countries = pm.country_list
countries.head()

In [ ]:
shapes = []
for index, row in countries.iterrows():
    update_progress(index / (len(countries)-1), 'Countries loaded')
    country = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_CODE', row.GAUL))
    country_json = geemap.ee_to_geojson(country)
    shapes.append(shape(country_json['features'][0]['geometry']))

In [ ]:
with rasterio.open(pm.llc_2000_raster) as src:
    out_image, out_transform = mask(src, shapes, all_touched=True)
    out_meta = src.meta.copy()
    out_meta.update(
        driver = 'GTiff',
        height = out_image.shape[1],
        width = out_image.shape[2],
        transform = out_transform,
        compress='lzw'
    )
    output = pm.get_result_dir() + "ESACCI-LC-L4-LCCS-Map-300m-P1Y-2000-v2.0.7_masked.tif"
    with rasterio.open(output, "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
with rasterio.open(pm.llc_2015_raster) as src:
    out_image, out_transform = mask(src, shapes, all_touched=True)
    out_meta = src.meta.copy()
    out_meta.update(
        driver = 'GTiff',
        height = out_image.shape[1],
        width = out_image.shape[2],
        transform = out_transform,
        compress='lzw'
    )
    output = pm.get_result_dir() + "ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7_masked.tif"
    with rasterio.open(output, "w", **out_meta) as dest:
        dest.write(out_image)